In [1]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import pdb
import os
os.chdir("/Users/guotianyu/GitHub/birth")

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple

from data import DataArgs, Dataset, iterate_batches
from ihead_full_model import ModelArgs, Transformer
%load_ext autoreload
%autoreload 2

In [11]:
ds = Dataset(DataArgs())

In [12]:
ds.num_tokens

67

In [15]:
x_exp, out_exp = ds.gen_batch(np.random.default_rng(0), 128)
x_exp

array([[65, 50, 39, ..., 54, 50, 43],
       [65, 58,  1, ..., 66, 47, 52],
       [65, 57, 47, ...,  1, 39, 63],
       ...,
       [65, 59, 57, ...,  6,  1, 47],
       [65,  6,  1, ..., 57, 53, 61],
       [65, 39, 56, ..., 46,  1, 51]])

In [14]:
np.where(x_exp == 66)

(array([  0,   0,   0, ..., 127, 127, 127]),
 array([ 27,  74,  78, ..., 232, 240, 251]))